<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/Text_generator_LSTM_0.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Generation With LSTM Recurrent Neural Networks in Python with Keras

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [1]:
!pip install konlpy

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import urllib.request
import nltk
nltk.download('punkt')

from konlpy.tag import Okt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    return txt 

In [3]:
# load ascii text and covert to lowercase

def get_sample_text(target_url):
    raw_text = urllib.request.urlopen(target_url).read().decode('utf-8')
    #raw_text = raw_text.lower()
    return nltk.sent_tokenize(clean_text(raw_text))

In [39]:
ko_sentences_dataset = []

In [21]:
ko_sentences_dataset += get_sample_text("https://raw.githubusercontent.com/dolmani38/Summary/master/data/korean_sample.txt")
# 여기서 계속 수집...
ko_sentences_dataset += get_sample_text("https://github.com/dolmani38/Summary/blob/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-ABC%20%EC%82%B4%EC%9D%B8%EC%82%AC%EA%B1%B4.txt")

In [117]:
org_text = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.
아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.
그러던 어느날, 소녀의 어머니가 병이들어 그만 세상을 떠나고 말았어요.
소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.
그래서 얼마 후 새어머니를 맞이했어요.
새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.
그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.
새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요.
그런데 이번에는 아버지마저 돌아가셨어요.
소녀는 하녀처럼 하루 종일 쓸고, 닦고, 집안일을 도맡아 했어요.
해도 해도 끝이 없는 집안일이 힘들어 지칠때면
난롯가에 앉아서 잠시 쉬곤 했지요.
"엄마, 저애를 신데렐라라고 불러야겠어요."
"온통 재투성이잖아요. 호호호!" 두 언니는 소녀를 놀려 댔어요.
어느 날, 왕궁에서 무도회가 열렸어요.
신데렐라의 집에도 초대장이 왔어요.
새어머니는 언니들을 데리고 무도회장으로 떠났어요.
신데렐라도 무도회에 가고 싶었어요.
혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.
"신데렐라, 너도 무도회에 가고 싶니?"
신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요.
"내가 너를 무도회에 보내주마 호박 한개와 생쥐 두마리, 도마뱀을 구해 오렴."
마법사 할머니가 주문을 외웠어요.
그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금 마차로 변했어요.
이번에는 생쥐와 도마뱀을 건드렸어요.
그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했답니다.
신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.
"신데렐라, 발을 내밀어 보거라."
할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요.
"신데렐라, 밤 열두시가 되면 모든게 처음대로 돌아간단다. 황금 마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼. 그러니까 반드시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지?"
왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.
왕자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고,신데렐라하고만 춤을 추었어요.
신데렐라는 왕자님과 춤을 추느라 시간 가는 줄도 몰랐어요.
땡, 땡, 땡...... 벽시계가 열두 시를 알리는 소리에 신데렐라는 화들짝 놀랐어요.
신데렐라가 허둥지둥 왕궁을 빠져나가는데,
유리 구두 한 짝이 벗겨졌어요.
하지만 구두를 주울 틈이 없었어요.
신데렐라를 뛰쫓아오던 왕자님은 층계에서 유리 구두 한 짝을 주웠어요.
왕자님은 유리 구두를 가지고 임금님께 가서 말했어요.
"이 유리 구두의 주인과 결혼하겠어요."
그래서 신하들은 유리 구두의 주인을 찾아 온 나라를 돌아다녔어요.
언니들은 발을 오므려도 보고, 구두를 늘려도 보았지만 한눈에 보기에도 유리 구두는 너무 작았어요.
그때, 신데렐라가 조용히 다가와 말했어요.
"저도 한번 신어 볼 수 있나요?"
신데렐라는 신하게 건넨 유리 구두를 신었어요,
유리 구두는 신데렐라의 발에 꼭 맞았어요.
신하들은 신데렐라를 왕궁으로 데리고 갔어요.
그 뒤 신데렐라는 왕자님과 결혼하여 오래오래 행복하게 살았대요.
"""

In [118]:
ko_sentences_dataset += [clean_text(org_text)]

In [7]:
len(ko_sentences_dataset)

1

In [119]:
text = ' '.join(ko_sentences_dataset)
text = text.strip()

In [9]:
len(text)

1516

In [120]:
text = clean_text(text)

In [121]:
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Corpus length: 2946
Total chars: 256
Number of sequences: 969


In [122]:
from tensorflow import keras
from tensorflow.keras import layers

import random
import io

model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(256,return_sequences=True),
        layers.Dropout(0.2),
        #layers.LSTM(256),
        #layers.Dropout(0.2),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
model.summary()
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_35 (LSTM)               (None, 40, 256)           525312    
_________________________________________________________________
dropout_30 (Dropout)         (None, 40, 256)           0         
_________________________________________________________________
lstm_36 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_20 (Dense)             (None, 256)               33024     
Total params: 755,456
Trainable params: 755,456
Non-trainable params: 0
_________________________________________________________________


In [123]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [124]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1000,verbose=0)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()


Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "싶니? 신데렐라가 고개를 들어보니 마법사 할머니가 빙그레 웃고 있었어요."
...Generated:   소녀에 신데 한 신씨사라이마하밤오 화개 시생 신두렐라 밤 뱀두렸 신데렐라도마말에 신아히  춤개 할혼게 예랬마은궁허리 시았어요. 그 뛰 에게 데 다 구두 시가 알었님나 신녀사이이 무계요. 변음 새 무도님장마지지했어요. 그 뛰궁보럼빠는 아느 시개개로 한리 구두 지아요 어머니가 놀아아요 울에 한고 신 들술 남아지보음회나 구두 오 뛰 아게 남에 소음초라 주나에요지 왕마에은이바바고라주 는 놀두간 뛰했자기. 돼번 무말에나 신아를 의 딸 하틈만라생마을까 아났렐라초게 데리로 늘날 왕아안에 구두 시두 보지지들셨 오두 주고고 잠들보까 왕돌님로 신하렐라하너행 신었간요뛰궁음뛰  아났오 말렸라부너 모이이오 울행틈어요 시두루 아혼로 왕했땡데 금 않마어느돼이렸도 왕했어요. 할진 다 짝짝를!변음뛰은 주녀어요. 신데들럼 유박 지두와 유

...Diversity: 0.5
...Generating with seed: "싶니? 신데렐라가 고개를 들어보니 마법사 할머니가 빙그레 웃고 있었어요."
...Generated:   소녀에 신데 한 신씨보라이마하밤오 화개 시생 신두렐라 밤 뱀두 구두 오에 마말 시고 호두 춤에 와고 갔느회까렸변음님과 뛰두 오느하 뛰고 뛰두를 유박 보음 밤렸요가 을 그에보.지데마왕도 아름로까요. 빙혼났까어기게 보도하재박 호두를 울 들에 말두 시아니은 유들 구두 !에 오에이 말렸럼도 유박 구밤 구두 구고 지두 틈궁보가 무도하바고 신두들라의 지한하로 왕했도 소녀가 되느뛰한 너났 갔짝는 유리 구두일까 왕했라요. 그 왕혼하두 구두 홀성 말음지잠 신두는 무이땅도 신고 라. 주마하니. 소번가 어요. 그데 다 홀나 주이 시갔 어기. 그 시두 구두를 무혼로 신니계 되느 오도 쳐들 호두 댔음와궁 힘녀새 과 되궁보요고 뛰두들 어하 왕렸님지여지홀여 추여보. 뛰

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...Generated:   잠를 쉬들해들 구두를 하마들애스난신데와라이게 반났 신음렐라의 모에로추발이이보 무혼땅셨간요. 저혼계라는 왕했어요. 그했는 왕마않은 맞렐 지하 은 할녀 볼에루 데 오두는 그 모가 잠느지쉬 면 딸에 오 니들 놀 저오께니유 개는는지얼났 자혼갔 소말렐 답이이지 바는는 려말 오 구고고운빠는이꼭 왕여님가 주마을고 아웠계 녔은 무 와아짝 쩍부에 구두 오 너리 투두와라가 딸 한말을오 너리 구두렐라니너 모꼭 오고 아말고 뛰말 구두를 신데들  밤셨어요 아스지 말고 갔여 소니머라놀밤 댔두를 뱀리로구건아라. 왕았않요 갔 새날님추게 아벽댔 힘아머니잖던 벗두이이갔 왕있못요. 그 오에 왕아님은 왕말님다 그리 구이간라돼게 왕두 므루와외웠행라세게 무도회자라지모하게 생가 다 딸도 말고 난데 왕두짝모뛰요 멋들 나요어 그 볼머데로 주데렐럼돼게


Generating text after epoch: 5
...Diversity: 0.2
...Generating with seed: "보다 나이가 위인 두 딸을 데리고 왔어요. 그러나 새어머니와 언니들은 성"
...Generated:   힘 가 한가 술지뛰도들한 신진 마도 울박들도 행에 볼생님들 왕아머니고 뛰데렐 가 온고이아간허고 왕두스오고 유했 벗했어요지는는 아개을 돼생 뱀리로 갔 한에 왕마님하두을 지데때 주마저잠. 잠느 반밤 돼두회에 왕리 오고 왔어요. 주마만 무고 한아요계 을셨는이이이계지유리 왕두뛰데 행고로자갔너 마일 오 을고 반리 구두를 무녀에 오 와마 아하로모지울리 추생이하루. 신데렐라. 그  말 한 갔 어기. 자혼하들고 의박 데 왕리 구고 오두 왕두니은 무도회에  아 힘개가 되요고 어요계 맞두 구았어요. 그두자갔갔행모어요. 그랬 말 되기지데씨들 나 왕리를 밤 그고 데 새날요도 뛰날렸힘생 새날요도 왕아님은 뛰아계춤. 그두를 보말렐 잠너과 왕마고 내리 구두 볼에 모에 모가로 니리 되두훌요도번뛰름 반드 이셨빙 가 오 왕아간고 뛰는는 

...Diversity: 0.5
...Generating with seed: "보다 나

KeyboardInterrupt: ignored

In [12]:
XXXXXX

NameError: ignored

In [42]:
okt = Okt()
word_set = []

for sentence in ko_sentences_dataset:
    word_set += [w for (w,m) in okt.pos(sentence)]

In [77]:
word_set = (' ' + clean_text(org_text) + ' ').split(' ')

In [78]:
# create mapping of unique chars to integers
word_to_int = dict((c, i) for i, c in enumerate(word_set))
word_to_int = dict((c, i) for i, c in enumerate(word_to_int))
int_to_word = dict((i, c) for i, c in enumerate(word_to_int))

In [79]:
n_words = len(word_set)
n_vocab = len(word_to_int)
print("Total Words: ", n_words)
print("Total Vocab: ", n_vocab)

Total Words:  342
Total Vocab:  277


In [80]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
	seq_in = word_set[i:i + seq_length]
	seq_out = word_set[i + seq_length]
	dataX.append([word_to_int[word] for word in seq_in])
	dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  332


In [81]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [94]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256*4, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 1024)              4202496   
_________________________________________________________________
dropout_17 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 277)               283925    
Total params: 4,486,421
Trainable params: 4,486,421
Non-trainable params: 0
_________________________________________________________________


In [106]:
#Larger LSTM Recurrent Neural Network

model = Sequential()
model.add(LSTM(256*2, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))

model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 10, 512)           1052672   
_________________________________________________________________
dropout_28 (Dropout)         (None, 10, 512)           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dropout_29 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 277)               71189     
Total params: 1,911,317
Trainable params: 1,911,317
Non-trainable params: 0
_________________________________________________________________


In [107]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [112]:
model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list,verbose=0)


Epoch 00001: loss did not improve from 0.88210

Epoch 00002: loss did not improve from 0.88210

Epoch 00003: loss did not improve from 0.88210

Epoch 00004: loss did not improve from 0.88210

Epoch 00005: loss did not improve from 0.88210

Epoch 00006: loss did not improve from 0.88210

Epoch 00007: loss did not improve from 0.88210

Epoch 00008: loss did not improve from 0.88210

Epoch 00009: loss did not improve from 0.88210

Epoch 00010: loss did not improve from 0.88210

Epoch 00011: loss did not improve from 0.88210

Epoch 00012: loss did not improve from 0.88210

Epoch 00013: loss did not improve from 0.88210

Epoch 00014: loss did not improve from 0.88210

Epoch 00015: loss did not improve from 0.88210

Epoch 00016: loss did not improve from 0.88210

Epoch 00017: loss did not improve from 0.88210

Epoch 00018: loss did not improve from 0.88210

Epoch 00019: loss did not improve from 0.88210

Epoch 00020: loss did not improve from 0.88210

Epoch 00021: loss did not improve from 

In [113]:
import numpy as np
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
print(start)
print(np.array(dataX).shape)
pattern = dataX[start]
print(pattern)
print("Seed:")
print(' '.join([int_to_word[value] for value in pattern]))

42
(332,)
[39, 40, 41, 42, 43, 44, 45, 46, 47, 34]
Seed:
딸을 데리고 왔어요. 그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요. 새어머니는


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [116]:
import sys
# generate characters
for i in range(10):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
    #index = sample(prediction)
	index = np.argmax(prediction)
	result = int_to_word[index]
	seq_in = [int_to_word[value] for value in pattern]
	sys.stdout.write(result+ ' ')
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

없는 없는 없는 집안일이 집안일이 집안일이 힘들어 힘들어 힘들어 지칠때면 
Done.
